In [16]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint as pprint 
import os
import pymongo
from copy import deepcopy

In [17]:
#Beginning Scrapping

In [18]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [19]:
base_url = 'https://www.payscale.com/cost-of-living-calculator/'
browser.visit(base_url)

In [20]:
base_html = browser.html
base_soup = bs(base_url, 'html.parser')

In [21]:
print(base_soup.prettify())

https://www.payscale.com/cost-of-living-calculator/



In [22]:
city_list = ['Washington-Seattle', 'Oregon-Portland', 'Arizona-Phoenix', 'Colorado-Denver']
table_dfs =[]
for city in city_list:
    df_name = city
    table_dfs.append(df_name)
    
table_dfs 

['Washington-Seattle', 'Oregon-Portland', 'Arizona-Phoenix', 'Colorado-Denver']

In [23]:
count = 0
pages = []
for page in range(2, 6):
    page_url = f"{base_url}{city_list[count]}"
    count = count + 1
    pages.append(page_url)
    print(page_url)    

https://www.payscale.com/cost-of-living-calculator/Washington-Seattle
https://www.payscale.com/cost-of-living-calculator/Oregon-Portland
https://www.payscale.com/cost-of-living-calculator/Arizona-Phoenix
https://www.payscale.com/cost-of-living-calculator/Colorado-Denver


In [24]:
print(pages)

['https://www.payscale.com/cost-of-living-calculator/Washington-Seattle', 'https://www.payscale.com/cost-of-living-calculator/Oregon-Portland', 'https://www.payscale.com/cost-of-living-calculator/Arizona-Phoenix', 'https://www.payscale.com/cost-of-living-calculator/Colorado-Denver']


In [26]:
headers = ['Median Home Price','Median Rent','Energy Bill','Phone Bill','Gas','Loaf of Bread', \
           'Gallon of Milk','Carton of Eggs','Bunch of Bananas','Hamburger',"Doctor's Visit",\
           'Dentist Visit','Optometrist Visit','Rx Drug','Veterinary Visit']
count = 0
lifestyle_df = pd.DataFrame()
for page in pages:
   browser.visit(page)
   western_region = browser.html
   western_soup = bs(western_region, 'html.parser')
   values = []
   for info in western_soup.find_all('div', class_='itemized__item-container'):
       for v in info.find_all('div', class_='itemized__item-value'):
           value = v.text
           values.append(deepcopy(value))
   style_df = pd.DataFrame([{headers[x]:values[x] for x in range(len(headers))}])
   lifestyle_df = lifestyle_df.append(deepcopy(style_df))
df = pd.DataFrame(lifestyle_df)
df['States']= city_list
western_data = df.set_index('States')
western_data

Bunch of Bananas Carton of Eggs Dentist Visit  \
States                                                             
Washington-Seattle            $4.26          $2.29       $118.13   
Oregon-Portland               $3.87          $2.09       $101.86   
Arizona-Phoenix               $3.27          $1.76        $92.92   
Colorado-Denver               $3.29          $1.77       $100.15   

                   Doctor's Visit       Energy Bill Gallon of Milk  \
States                                                               
Washington-Seattle        $137.50  $206.39 / month           $2.46   
Oregon-Portland           $118.57  $130.77 / month           $2.23   
Arizona-Phoenix           $108.16  $161.12 / month           $1.89   
Colorado-Denver           $116.58  $156.76 / month           $1.90   

                                Gas Hamburger Loaf of Bread Median Home Price  \
States                                                                          
Washington-Seattle  $3.47 / gallon      $5.37         $4.22          $673,874   
Oregon-Portland     $2.78 / gallon      $4.88         $3.84          $620,783   
Arizona-Phoenix     $2.39 / gallon      $4.13         $3.24          $330,344   
Colorado-Denver     $2.75 / gallon      $4.15         $3.26          $463,245   

                        Median Rent Optometrist Visit        Phone Bill  \
States                                                                    
Washington-Seattle  $2,111 / month            $128.85  $220.00 / month    
Oregon-Portland     $1,945 / month            $111.11  $139.40 / month    
Arizona-Phoenix     $1,035 / month            $101.35  $171.75 / month    
Colorado-Denver     $1,451 / month            $109.24  $167.10 / month    

                    Rx Drug Veterinary Visit  
States                                        
Washington-Seattle  $547.72           $69.64  
Oregon-Portland     $472.31           $60.53  
Arizona-Phoenix     $430.86           $48.23  
Colorado-Denver     $464.37           $55.28

In [29]:
for x in western_data:
   western_data[x] = western_data[x].str.replace("$","")
   western_data[x] = western_data[x].str.replace("month","")
   western_data[x] = western_data[x].str.replace("/","")
   western_data[x] = western_data[x].str.replace("gallon","")
   western_data[x] = western_data[x].str.replace(",","")
   western_data[x] = pd.to_numeric(western_data[x])
western_data.head()

Bunch of Bananas  Carton of Eggs  Dentist Visit  \
States                                                                
Washington-Seattle              4.26            2.29         118.13   
Oregon-Portland                 3.87            2.09         101.86   
Arizona-Phoenix                 3.27            1.76          92.92   
Colorado-Denver                 3.29            1.77         100.15   

                    Doctor's Visit  Energy Bill  Gallon of Milk   Gas  \
States                                                                  
Washington-Seattle          137.50       206.39            2.46  3.47   
Oregon-Portland             118.57       130.77            2.23  2.78   
Arizona-Phoenix             108.16       161.12            1.89  2.39   
Colorado-Denver             116.58       156.76            1.90  2.75   

                    Hamburger  Loaf of Bread  Median Home Price  Median Rent  \
States                                                                         
Washington-Seattle       5.37           4.22             673874         2111   
Oregon-Portland          4.88           3.84             620783         1945   
Arizona-Phoenix          4.13           3.24             330344         1035   
Colorado-Denver          4.15           3.26             463245         1451   

                    Optometrist Visit  Phone Bill  Rx Drug  Veterinary Visit  
States                                                                        
Washington-Seattle             128.85      220.00   547.72             69.64  
Oregon-Portland                111.11      139.40   472.31             60.53  
Arizona-Phoenix                101.35      171.75   430.86             48.23  
Colorado-Denver                109.24      167.10   464.37             55.28

In [33]:
western_data.mean(axis=0)

Bunch of Bananas          3.6725
Carton of Eggs            1.9775
Dentist Visit           103.2650
Doctor's Visit          120.2025
Energy Bill             163.7600
Gallon of Milk            2.1200
Gas                       2.8475
Hamburger                 4.6325
Loaf of Bread             3.6400
Median Home Price    522061.5000
Median Rent            1635.5000
Optometrist Visit       112.6375
Phone Bill              174.5625
Rx Drug                 478.8150
Veterinary Visit         58.4200
dtype: float64

In [36]:
region = pd.DataFrame(western_data.mean().round(2))
region_t = region.T
region.T #This is tranposing the data

Bunch of Bananas  Carton of Eggs  Dentist Visit  Doctor's Visit  \
0              3.67            1.98         103.27           120.2   

   Energy Bill  Gallon of Milk   Gas  Hamburger  Loaf of Bread  \
0       163.76            2.12  2.85       4.63           3.64   

   Median Home Price  Median Rent  Optometrist Visit  Phone Bill  Rx Drug  \
0           522061.5       1635.5             112.64      174.56   478.81   

   Veterinary Visit  
0             58.42

In [37]:
region_n = "Western Region"
the_df = pd.DataFrame(region_t)
the_df['Region'] = region_n

western_data_index = the_df.set_index('Region')
western_data_index

Bunch of Bananas  Carton of Eggs  Dentist Visit  \
Region                                                            
Western Region              3.67            1.98         103.27   

                Doctor's Visit  Energy Bill  Gallon of Milk   Gas  Hamburger  \
Region                                                                         
Western Region           120.2       163.76            2.12  2.85       4.63   

                Loaf of Bread  Median Home Price  Median Rent  \
Region                                                          
Western Region           3.64           522061.5       1635.5   

                Optometrist Visit  Phone Bill  Rx Drug  Veterinary Visit  
Region                                                                    
Western Region             112.64      174.56   478.81             58.42